## Import libraries

In [187]:
# import libraries

import datetime
import pickle
import nltk
import re
import scipy
import pytz
import pandas as pd
from scipy import sparse
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from utils import get_weather
from utils import text_processing

## Get and preprocess ISW files

In [188]:
# get article from yesterday

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

yesterday_day = yesterday.day
yesterday_month = yesterday.month
yesterday_year = yesterday.year

In [189]:
file = text_processing.get_article_from_yesterday(yesterday_day,yesterday_month,yesterday_year)

In [190]:
data = text_processing.read_html(file)

In [191]:
def preprocess_all_text(data):
    pattern = "\[(\d+)\]"
    data['main_html_v1'] = data['main_html'].apply(lambda x: re.sub(pattern,"",str(x)))
    data['main_html_v2'] = data['main_html_v1'].apply(lambda x: re.sub(r'http(\S+.*\s)',"",x))
    data['main_html_v3'] = data['main_html_v2'].apply(lambda x: re.sub(r'2022|2023|©2022|©2023|\xa0|\n',"",x))
    data['main_html_v4'] = data['main_html_v3'].apply(lambda x: BeautifulSoup(x).text)
    data['main_html_v5'] = data['main_html_v4'].apply(lambda x: text_processing.remove_names_and_dates(x))
    
    return data

In [192]:
data_preprocessed = preprocess_all_text(data)

In [193]:
data_preprocessed = data_preprocessed.drop(['main_html_v1','main_html_v2','main_html_v3','main_html_v4'],axis=1)

In [194]:
# Uncommit for first time download

# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

In [195]:
# lemmatization and stemming algorithms to get the unique words

data_preprocessed['report_text_lemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"lemm"))
data_preprocessed['report_text_stemm'] = data_preprocessed['main_html_v5'].apply(lambda x: text_processing.preprocess(x,"stemm"))

In [196]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm
0,2023-05-02,russian_offensive_campaign_assessment_May_2_2023,"Russian Offensive Campaign Assessment, May 2, ...","Russian Offensive Campaign Assessment, May 2, ...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, May 2,...",russian offens campaign ass may riley bailey ...,russian offen campaign assess may riley baile...


In [197]:
docs = data_preprocessed['report_text_lemm'].tolist()

## ISW vectorize

In [198]:
# apply vectorizing and tf-idf algorithm

cv = CountVectorizer()
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

tf_idf_vector = tfidf_transformer.transform(word_count_vector)

In [199]:
tf_idf_vector

<1x714 sparse matrix of type '<class 'numpy.float64'>'
	with 714 stored elements in Compressed Sparse Row format>

In [200]:
# extract the most valuable words in article

feature_names = cv.get_feature_names_out()
tf_idf_vector
data_preprocessed['keywords'] = data_preprocessed['report_text_stemm'].apply(lambda x: text_processing.convert_doc_to_vector(x,feature_names,tf_idf_vector))

In [201]:
data_preprocessed['keywords'][0]

{'russian': 0.686,
 'ukrainian': 0.332,
 'forc': 0.26,
 'claim': 0.188,
 'may': 0.144,
 'not': 0.116,
 'oblast': 0.111,
 'conduct': 0.111,
 'report': 0.105,
 'defens': 0.094}

In [202]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords
0,2023-05-02,russian_offensive_campaign_assessment_May_2_2023,"Russian Offensive Campaign Assessment, May 2, ...","Russian Offensive Campaign Assessment, May 2, ...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, May 2,...",russian offens campaign ass may riley bailey ...,russian offen campaign assess may riley baile...,"{'russian': 0.686, 'ukrainian': 0.332, 'forc':..."


## Part of script: Final preprocessing and merging

In [203]:
data_preprocessed["date_datetime"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed['date_tomorrow_datetime'] = data_preprocessed['date_datetime'].apply(lambda x: x+datetime.timedelta(days=1))
data_preprocessed = data_preprocessed.rename(columns = {"date_datetime":"report_date"})

In [204]:
data_preprocessed.head()

,date,short_url,title,text_title,full_url,main_html,main_html_v5,report_text_lemm,report_text_stemm,keywords,report_date,date_tomorrow_datetime
0,2023-05-02,russian_offensive_campaign_assessment_May_2_2023,"Russian Offensive Campaign Assessment, May 2, ...","Russian Offensive Campaign Assessment, May 2, ...",/backgrounder/russian-offensive-campaign-asses...,"[[[ , <p align=""center"" style=""text-align: lef...","Russian Offensive Campaign Assessment, May 2,...",russian offens campaign ass may riley bailey ...,russian offen campaign assess may riley baile...,"{'russian': 0.686, 'ukrainian': 0.332, 'forc':...",2023-05-02,2023-05-03


In [205]:
data_vectorised = tf_idf_vector.toarray()
vectors_df = pd.DataFrame(data_vectorised)
vectors_df['date'] = pd.to_datetime(today)


In [206]:
vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,705,706,707,708,709,710,711,712,713,date
0,0.005536,0.005536,0.011072,0.005536,0.011072,0.011072,0.005536,0.011072,0.005536,0.016608,...,0.005536,0.005536,0.011072,0.005536,0.005536,0.02768,0.005536,0.022144,0.005536,2023-05-03


In [207]:
df_isw_short = data_preprocessed[['date','report_text_lemm','keywords','date_tomorrow_datetime']]

In [208]:
df_isw_short.head()

,date,report_text_lemm,keywords,date_tomorrow_datetime
0,2023-05-02,russian offens campaign ass may riley bailey ...,"{'russian': 0.686, 'ukrainian': 0.332, 'forc':...",2023-05-03


## Get weather and make predictions for all regions

In [209]:
# load necessary models

tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))
model = pickle.load(open("models/training_models/4_rf_3.1f.pkl","rb"))

C:\Users\dimai\AppData\Local\Temp\ipykernel_12068\660841987.py:3: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
C:\Users\dimai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\dimai\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.24.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.

In [210]:
cities = ['Kyiv', 'Kharkiv', 'Lviv', 'Odesa', 'Donetsk', 'Luhansk', 'Kherson', 'Simferopol', 'Mykolaiv', 'Lutsk', 'Chernihiv', 'Dnipro', 'Vinnytsia', 'Zhytomyr', 'Kropyvnytskyi', 'Poltava', 'Sumy', 'Rivne', 'Khmelnytskyi', 'Chernivtsi', 'Ternopil', 'Cherkasy', 'Uzhgorod', 'Zaporozhye', 'Ivano-Frankivsk']

date = datetime.datetime.now(pytz.timezone('Europe/Kyiv'))
result = {}

# for loop generate weather, merged it with isw vector and make a prediction for all regions in list above

for city in cities:

    df_weather_complete = get_weather.get_weather_for_12_hours(city,date)

    # merge
    df_weather_complete['key']=1
    df_isw_short['key']=1
    df_all = df_weather_complete.merge(df_isw_short, how = 'left', left_on = 'key', right_on = 'key')

    # drop
    to_drop=['key','date','date_tomorrow_datetime','keywords','report_text_lemm']
    if 'sunrise' in df_all.columns:
        exceptions = ['sunset','sunrise']
        to_drop.extend(exceptions)
    df_weather_matrix = df_all.drop(to_drop, axis = 1)

    # final dataset
    df_weather_matrix['Unnamed: 0'] = 0
    df_weather_matrix= df_weather_matrix[['Unnamed: 0', 'day_datetimeEpoch', 'day_tempmax', 'day_tempmin', 'day_temp', 'day_dew', 'day_humidity', 'day_precip', 'day_precipcover', 'day_solarradiation', 'day_solarenergy', 'day_uvindex', 'day_moonphase', 'hour_datetimeEpoch', 'hour_temp', 'hour_humidity', 'hour_dew', 'hour_precipprob', 'hour_snow', 'hour_snowdepth', 'hour_windgust', 'hour_windspeed', 'hour_winddir', 'hour_pressure', 'hour_visibility', 'hour_cloudcover', 'hour_severerisk', 'region_id']]

    cv_vector_model = cv.transform(df_all['report_text_lemm'].values.astype('U'))
    tf_idf_model = tfidf.transform(cv_vector_model)

    # merge final dataset with tfidf vector
    df_weather_matrix_csr = scipy.sparse.csr_matrix(df_weather_matrix)
    df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_csr, tf_idf_model), format='csr')

    #predict
    predicted = model.predict(df_all_data_csr)
    hours = []
    
    # store results in dictionary, after extract 
    for i in range(12):
        hour = date + datetime.timedelta(hours=i)
        hour_rounded = hour.replace(minute=0, second=0, microsecond=0)
        hours.append(hour_rounded.strftime('%Y-%m-%d %H:%M'))

    result[city] = dict(zip(hours, predicted))

#Show result
result

C:\Users\dimai\Documents\Study\Python\Alert-Prediction\utils\get_weather.py:79: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\dimai\Documents\Study\Python\Alert-Prediction\utils\get_weather.py:79: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_weather_hours['hour_int']=pd.to_datetime(df_weather_hours['hour_datetime']).dt.hour
C:\Users\dimai\AppData\Local\Temp\ipykernel_12068\2919423775.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

{'Kyiv': {'2023-05-03 15:00': 0,
  '2023-05-03 16:00': 0,
  '2023-05-03 17:00': 0,
  '2023-05-03 18:00': 0,
  '2023-05-03 19:00': 0,
  '2023-05-03 20:00': 0,
  '2023-05-03 21:00': 0,
  '2023-05-03 22:00': 0,
  '2023-05-03 23:00': 0,
  '2023-05-04 00:00': 0,
  '2023-05-04 01:00': 0,
  '2023-05-04 02:00': 0},
 'Kharkiv': {'2023-05-03 15:00': 0,
  '2023-05-03 16:00': 0,
  '2023-05-03 17:00': 0,
  '2023-05-03 18:00': 0,
  '2023-05-03 19:00': 0,
  '2023-05-03 20:00': 0,
  '2023-05-03 21:00': 0,
  '2023-05-03 22:00': 0,
  '2023-05-03 23:00': 0,
  '2023-05-04 00:00': 0,
  '2023-05-04 01:00': 0,
  '2023-05-04 02:00': 0},
 'Lviv': {'2023-05-03 15:00': 0,
  '2023-05-03 16:00': 0,
  '2023-05-03 17:00': 0,
  '2023-05-03 18:00': 0,
  '2023-05-03 19:00': 0,
  '2023-05-03 20:00': 0,
  '2023-05-03 21:00': 0,
  '2023-05-03 22:00': 0,
  '2023-05-03 23:00': 0,
  '2023-05-04 00:00': 0,
  '2023-05-04 01:00': 0,
  '2023-05-04 02:00': 0},
 'Odesa': {'2023-05-03 15:00': 0,
  '2023-05-03 16:00': 0,
  '2023-05-

##     Save results

In [211]:
result = pd.DataFrame(result)
VERSION = "3"
result.to_csv(f'data/results/results_{VERSION}.txt', sep='\t', index=False)